No. 5 Time Series

In [139]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf # Deep learning library
import matplotlib.pyplot as plt # Visualization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [140]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [141]:
df = pd.read_csv('/content/drive/MyDrive/Final Exam - Tipe B/Question 5/Cryptocurrency2020.csv')
df

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume
0,1.618880e+12,4/20/2021 0:02,BTCUSD,55717.47,55723.00,55541.69,55541.69,2.406734
1,1.618880e+12,4/20/2021 0:01,BTCUSD,55768.94,55849.82,55711.74,55717.47,0.573458
2,1.618880e+12,4/20/2021 0:00,BTCUSD,55691.79,55793.15,55691.79,55768.94,3.309904
3,1.618880e+12,4/19/2021 23:59,BTCUSD,55777.86,55777.86,55677.92,55691.79,0.684934
4,1.618880e+12,4/19/2021 23:58,BTCUSD,55803.50,55823.88,55773.08,55777.86,0.477225
...,...,...,...,...,...,...,...,...
658060,1.577840e+12,1/1/2020 0:04,BTCUSD,7163.60,7163.60,7163.00,7163.00,1.751844
658061,1.577840e+12,1/1/2020 0:03,BTCUSD,7163.00,7163.60,7163.00,7163.60,0.002706
658062,1.577840e+12,1/1/2020 0:02,BTCUSD,7163.30,7164.22,7163.00,7163.00,0.063904
658063,1.577840e+12,1/1/2020 0:01,BTCUSD,7163.30,7164.23,7163.30,7163.30,0.002646


In [142]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    if shuffle_buffer > 0:
        ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[-1:]))
    return ds.batch(batch_size).prefetch(1)

train, val = train_test_split(df.iloc[:, 3:], test_size=0.2, shuffle=False)
scaler = MinMaxScaler().fit(train)
train = scaler.transform(train)[:, 0]
val = scaler.transform(val)[:, 0]
train = windowed_dataset(train, window_size=64, batch_size=128, shuffle_buffer=1000)
val = windowed_dataset(val, window_size=64, batch_size=128, shuffle_buffer=1000)

In [147]:
#Buatlah model dengan input layer disarankan menggunakan LSTM LSTM dan output layer 1 

model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(units=30, input_shape=[None, 1]),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Activation("linear"),
])

In [148]:
#gunakan loss MAE, optimizer boleh bebas memilih
model.compile(
    optimizer='adam', loss='mae'
)

In [149]:
callbacks=[
        tf.keras.callbacks.EarlyStopping(
            patience=10, 
            restore_best_weights=True
        ), 
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            patience=5
        )
    ]

In [151]:
#silakan menentukan banyaknya epoch sendiri
# Biar cantik aja progress bar nya
import sys
!{sys.executable} -m pip install tensorflow-addons
import tensorflow_addons as tfa
tqdm_callback = tfa.callbacks.TQDMProgressBar()

history = model.fit(
    train, validation_data=val, epochs=20, steps_per_epoch=25,verbose=0, callbacks=[callbacks,tqdm_callback], shuffle=True
)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Training:   0%|           0/20 ETA: ?s,  ?epochs/s

Epoch 1/20


0/25           ETA: ?s - 

Epoch 2/20


...

0/25           ETA: ?s - 

Epoch 19/20


0/25           ETA: ?s - 

Epoch 20/20


0/25           ETA: ?s - 

In [153]:
#save model menjadi .h5
model.save('model5.h5')